## Introduction 

This notebook will go over a new loss function to optimize the decision of the ERUPT metric directly for both single and mutliple response case.

1. New Loss Function for single response variable 

2. Building / Gridsearching the Uplift Model with new loss vs default mean-squared-error loss

3. Extending new loss function to multi-responses 



### New Loss Function  

Readers are encouraged to read the [example on single responses](https://github.com/Ibotta/ibotta_uplift/blob/master/examples/mr_uplift_one_response_example.ipynb) for a more thorough background on uplift models and the ERUPT metric. 

To recap we start with the decision function defined as: 

\begin{equation}
 \pi(x_i) =argmax \:_{t \in T} E[y_i | X=x_i, T=t]
\tag{1}
\end{equation}

This is the function that determines whether to assign a user a particular treatment for a single response we wish to maximize. It just says that we apply the treatment with the highest expected value.

To exaluate the decision function the ERUPT metric is used. It estimates the expected value of the decision function on an out-of-sample dataset. Assuming that the treatments are uniformly (all treatments have same number of observations) and randomly assigned the ERUPT metric is:

$$ERUPT(pi(x),y,t) \propto \sum_{i}^N y_i I(\pi(x_i) = t_i) \tag{2}$$



The original loss functions used in mr_uplift have used a mean squared error (MSE) function $f(x,t)$ to estimate $E[y_i | X=x_i, T=t]$ :


\begin{equation}
 MSE(y,f(x,t)) = 1/N \sum_{i}^N (y_i-f(X=x_i, T=t_i))^2
\tag{3}
\end{equation}


In theory this should produce an estimator that approximates the interaction between $x$ and $t$ neccesary to build a good decision function $\pi(x_i)$. However, in practice thie interaction effect is tricky to estimate because the it is generally small relative to main effects and noise in the response. Several researchers have developed in techniques order to estimate this (see [Zhao, Fang, Simchi-Levi](https://arxiv.org/pdf/1705.08492.pdf?source=post_page---------------------------), [Athey, Tibshirani, Wager](https://arxiv.org/pdf/1610.01271.pdf), [Hitsch, Misra](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3111957) to name a few).


The approach taken here follows an approach developed by [Weiss](https://medium.com/building-ibotta/maximizing-the-erupt-metric-for-uplift-models-f8d7e57bfdf2) and [Fernandez-Loria, et al](https://arxiv.org/pdf/2004.11532.pdf) in which it attempts to optimize the ERUPT function in $2$ directly. 

In order to approximate that function we assign a 'probability' score to each treatment: 


\begin{equation}
 optimizedloss(y,t,f(t,x)) =  \sum_{i}^N \sum_{t \in T}^T y_{i,t}*p(t|x_i)  =  \sum_{i}^N \sum_{t \in T}^T y_{i,t}*\frac{e^{f(X=x_i, T=t)}}{\sum_{t \in T}e^{f(X=x_i, T=t)}}
\tag{4}
\end{equation}
where $y_{i_t}$ is equal to $y_{i}$ if user $i$ recieved treatment $t$, else zero. 

One major difference here compared with [Weiss](https://medium.com/building-ibotta/maximizing-the-erupt-metric-for-uplift-models-f8d7e57bfdf2) is that it explitly incorporates the treatments in the function $f()$ to be estimated. This should help regularize the estimation if the treatments have similar meta-data information. For instance; if the treatments are ordered then this model can take that information into account.

Finally, MSE is also included in the loss function to create a weighted loss between the two. Including MSE should help when the estimated optimal treatment does not equal the treatment assignment:

\begin{equation}
\alpha*optimizedloss(y,t,f(t,x)) + (1-\alpha)*MSE(y,f(x,t))
\tag{5}
\end{equation}




In [1]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)


/Library/Frameworks/Python.framework/Versions/3.8/bin/python3
3.8.7 (v3.8.7:6503f05dd5, Dec 21 2020, 12:45:15) 
[Clang 6.0 (clang-600.0.57)]
sys.version_info(major=3, minor=8, micro=7, releaselevel='final', serial=0)


In [2]:
#from ggplot import *
import numpy as np
import pandas as pd

from mr_uplift.dataset.data_simulation import get_sin_uplift_data
from mr_uplift.mr_uplift import MRUplift


### Data Generating Process

In order to test the new loss function I simulated some data and compared results with a regular MSE loss. Below is the data generating process of the data. This is similar to the previous data function except that the treatment effect relationship is now in a sine function. This adds a layer of complexity for the model to estimate.

\begin{equation}
x_1  \sim runif(0,1)
\end{equation}

\begin{equation}
x_2 \sim runif(0,1)
\end{equation}

\begin{equation}
e_1 \sim rnorm(0,1)
\end{equation}

\begin{equation}
e_2 \sim rnorm(0,1)
\end{equation}

\begin{equation}
t \sim rbinom(.5)
\end{equation}

\begin{equation}
noise \sim rnorm(0,1)
\end{equation}

\begin{equation}
revenue = sin(20*x_1*t) + e_1
\end{equation}

\begin{equation}
costs = sin(20*x_2*t) + e_2
\end{equation}

\begin{equation}
profit = revenue - costs
\end{equation}




In [3]:
np.random.seed(23)

y, x, t = get_sin_uplift_data(10000)

y = pd.DataFrame(y)
y.columns = ['revenue','cost', 'noise']
y['profit'] = y['revenue'] - y['cost']

In [7]:
#Build Uplift with the default MSE Loss
uplift_model = MRUplift()
param_grid = dict(num_nodes=[8, 32], dropout=[.1,.25], activation=[
                          'relu'], num_layers=[2,3], epochs=[25], batch_size=[100])



uplift_model.fit(x, np.array(y['profit']).reshape(-1,1), t.reshape(-1,1), param_grid = param_grid, n_jobs = 1, cv = 5)

#Build Uplift with the Optimization Loss 
#alpha and copy_several_times are new hyperparameters. alpha is the weight given to MSE vs optimi_loss from eqn (5)
#copy_several_times is for multiple response uplift models and is described below
uplift_model_optimized = MRUplift()
param_grid_optimized = dict(num_nodes=[8, 32], dropout=[.1,.25], activation=[
                          'relu'], num_layers=[2,3], epochs=[25], batch_size=[100],
                 copy_several_times = [None], 
                 alpha = [.999,.99,.75,.5])
uplift_model_optimized.fit(x, np.array(y['profit']).reshape(-1,1), t.reshape(-1,1),
                           param_grid = param_grid_optimized, n_jobs = 1, optimized_loss = True, cv=5)
uplift_model_optimized_propensity = MRUplift()
param_grid_optimized = dict(num_nodes=[8, 32], dropout=[.1,.25], activation=[
                          'relu'], num_layers=[2,3], epochs=[25], batch_size=[100],
                 copy_several_times = [None], 
                 alpha = [.999,.99,.75,.5])
uplift_model_optimized_propensity.fit(x, np.array(y['profit']).reshape(-1,1), t.reshape(-1,1),
                           param_grid = param_grid_optimized, n_jobs = 1, optimized_loss = True, cv=5,
                           use_propensity = True)


The following Variables were used a Lambda layer's call (tf.tile_822), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_823), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_824), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a s

The following Variables were used a Lambda layer's call (tf.tile_832), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_833), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_834), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a s

The following Variables were used a Lambda layer's call (tf.tile_842), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_843), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_844), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a s

The following Variables were used a Lambda layer's call (tf.tile_852), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_853), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_854), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a s

The following Variables were used a Lambda layer's call (tf.tile_862), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_863), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_864), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a s

...

/Users/sweiss/Documents/GitHub/mr_uplift/mr_uplift/keras_model_functionality.py:383: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  weights_test = propensity_scores_test.lookup(str_t_test_series.index, str_t_test_series.values)
/Users/sweiss/Documents/GitHub/mr_uplift/mr_uplift/keras_model_functionality.py:390: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  weights_train = propensity_scores_train.lookup(str_t_train_series.index, str_t_train_series.values)


The following Variables were used a Lambda layer's call (tf.tile_1462), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_1463), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


/Users/sweiss/Documents/GitHub/mr_uplift/mr_uplift/keras_model_functionality.py:383: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  weights_test = propensity_scores_test.lookup(str_t_test_series.index, str_t_test_series.values)
/Users/sweiss/Documents/GitHub/mr_uplift/mr_uplift/keras_model_functionality.py:390: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  weights_train = propensity_scores_train.lookup(str_t_train_series.index, str_t_train_series.values)


The following Variables were used a Lambda layer's call (tf.tile_1464), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_1465), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


/Users/sweiss/Documents/GitHub/mr_uplift/mr_uplift/keras_model_functionality.py:454: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  weights_train = propensity_scores_train.lookup(str_t_train_series.index, str_t_train_series.values)


### Results MSE vs Optimiziation 

The results below show the MSE only loss function increases profits by around ~0.04 while the new optimization results in profit score of ~0.17. This shows the new loss function can help estimate the heterogeneous effect more effectively in this example.  

In [8]:
erupt_curves_mse, distribution_mse = uplift_model.get_erupt_curves()
print('ERUPT Metric for MSE Model')
print(erupt_curves_mse)

print('Distribution of Treatments for MSE Model')
print(distribution_mse)

print('Best Parameters MSE')
print(uplift_model.best_params_net)


erupt_curves_optimized, distribution_optimized = uplift_model_optimized.get_erupt_curves()

print('ERUPT Metric for Optimized Model')
print(erupt_curves_optimized)

print('Distribution of Treatments for Optimized Model')
print(distribution_optimized)

print('Best Parameters Optimized')
print(uplift_model_optimized.best_params_net)


erupt_curves_optimized_propensity, distribution_optimized_propensity = uplift_model_optimized_propensity.get_erupt_curves()

print('ERUPT Metric for Optimized Model')
print(erupt_curves_optimized_propensity)

print('Distribution of Treatments for Optimized Model')
print(distribution_optimized_propensity)

print('Best Parameters Optimized')
print(uplift_model_optimized_propensity.best_params_net)


Using Test Data Set
ERUPT Metric for MSE Model
       mean       std response_var_names weights assignment treatment
0  0.022245  0.007116              var_0       1      model        -1
0  0.000367  0.007414              var_0       1     random        -1
0  0.001738  0.002379              var_0      -1        ate       [0]
0 -0.017890  0.017292              var_0      -1        ate       [1]
Distribution of Treatments for MSE Model
   num_observations  tmt weights  percent_tmt
0              5705    0       1        0.815
1              1295    1       1        0.185
Best Parameters MSE
{'activation': 'relu', 'batch_size': 100, 'dropout': 0.25, 'epochs': 25, 'num_layers': 3, 'num_nodes': 8}
Using Test Data Set
ERUPT Metric for Optimized Model
       mean       std response_var_names weights assignment treatment
0  0.218722  0.012435              var_0       1      model        -1
0  0.008585  0.013221              var_0       1     random        -1
0  0.001738  0.002379              

### Multi-Response Optimization

See  [example on multiple_responses](https://github.com/Ibotta/ibotta_uplift/blob/master/examples/mr_uplift_multiple_response_example.ipynb) for a more detailed introduction on uplift models with multiple responses.


The general idea of uplift models with multiple response variables is to create a new variable that is a weighted sum of the responses. These weights define the relative importance responses variables. One can then calculate ERUPT metrics for a given set of weights as if it was a single response. 

However, issue of extending the loss function from $(4)$ to a multi-response framework is that it only optimizes one set of weights. Ideally we'd like maximize a function that 'integrates out' the weights. Instead of doing that explitily I draw a random uniform weight for each observation-response pair and maximize the following:

\begin{equation}
w_{i,j} \sim runif(-1,1)
\end{equation}

\begin{equation}
 optimizedlossmultioutput(y,t,w,f(t,x)) = \sum_{i}^N \sum_{t \in T}^T (\sum_{j}^J w_{i,j} y_{i,t,j})*\frac{e^{\sum_{j}^J w_{i,j} f_j(X=x_i, T=t)}}{\sum_{t \in T}e^{\sum_{j}^J w_{i,j} f_j(X=x_i, T=t)}}
\tag{6}
\end{equation}

Note that the weights $w_{i,j}$ do not explicitly go into the function $f(t,x)$. 

Different 'optimal treatments' will be selected due to randomness of the weights for two identical observations. Because of this I included another hyperparameter 'copy_several_times' that duplicates observations a specified number of times. I've found this to be helpful in practice

An MSE loss is also include similar to $(5)$. This has two main benefits 1) to create help estimate the function when the optimal treatment was not randomly assigned and 2) to maintain the order of response predictions. Becuase of these reasons I would not set $\alpha$ to be 1. 

Below is an example of the multi-response case.


In [ ]:
uplift_model_optimized_mr = MRUplift()
param_grid = dict(num_nodes=[ 32,64, 256], dropout=[.1], activation=[
                          'relu'], num_layers=[1,2,3], epochs=[30], batch_size=[100],
                 copy_several_times = [1], 
                 alpha = [.999,.99,.5,.25,0])

uplift_model_optimized_mr.fit(x, y, t.reshape(-1,1),
                           param_grid = param_grid, n_jobs = 1, optimized_loss = True, cv=5,
                             use_propensity = True )


The following Variables were used a Lambda layer's call (tf.tile_1918), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_1919), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


/Users/sweiss/Documents/GitHub/mr_uplift/mr_uplift/keras_model_functionality.py:383: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  weights_test = propensity_scores_test.lookup(str_t_test_series.index, str_t_test_series.values)
/Users/sweiss/Documents/GitHub/mr_uplift/mr_uplift/keras_model_functionality.py:390: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  weights_train = propensity_scores_train.lookup(str_t_train_series.index, str_t_train_series.values)


The following Variables were used a Lambda layer's call (tf.tile_1920), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_1921), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


/Users/sweiss/Documents/GitHub/mr_uplift/mr_uplift/keras_model_functionality.py:383: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  weights_test = propensity_scores_test.lookup(str_t_test_series.index, str_t_test_series.values)
/Users/sweiss/Documents/GitHub/mr_uplift/mr_uplift/keras_model_functionality.py:390: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  weights_train = propensity_scores_train.lookup(str_t_train_series.index, str_t_train_series.values)


The following Variables were used a Lambda layer's call (tf.tile_1922), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_1923), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


...

/Users/sweiss/Documents/GitHub/mr_uplift/mr_uplift/keras_model_functionality.py:383: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  weights_test = propensity_scores_test.lookup(str_t_test_series.index, str_t_test_series.values)
/Users/sweiss/Documents/GitHub/mr_uplift/mr_uplift/keras_model_functionality.py:390: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  weights_train = propensity_scores_train.lookup(str_t_train_series.index, str_t_train_series.values)


The following Variables were used a Lambda layer's call (tf.tile_2177), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_2178), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_2179), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is 

/Users/sweiss/Documents/GitHub/mr_uplift/mr_uplift/keras_model_functionality.py:383: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  weights_test = propensity_scores_test.lookup(str_t_test_series.index, str_t_test_series.values)
/Users/sweiss/Documents/GitHub/mr_uplift/mr_uplift/keras_model_functionality.py:390: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  weights_train = propensity_scores_train.lookup(str_t_train_series.index, str_t_train_series.values)


The following Variables were used a Lambda layer's call (tf.tile_2180), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tile_2181), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


/Users/sweiss/Documents/GitHub/mr_uplift/mr_uplift/keras_model_functionality.py:383: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  weights_test = propensity_scores_test.lookup(str_t_test_series.index, str_t_test_series.values)
/Users/sweiss/Documents/GitHub/mr_uplift/mr_uplift/keras_model_functionality.py:390: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  weights_train = propensity_scores_train.lookup(str_t_train_series.index, str_t_train_series.values)


In [10]:
erupt_curves_optimized_mr, distribution_optimized_mr = uplift_model_optimized_mr.get_erupt_curves()

Using Test Data Set


In [ ]:
from ggplot import *

distribution_optimized_mr['weights_1'] = [np.float(x.split(',')[0]) for x in distribution_optimized_mr['weights']]
erupt_curves_optimized_mr['weights_1'] = [np.float(x.split(',')[0]) for x in erupt_curves_optimized_mr['weights']]

ggplot(aes(x='weights_1', y='mean', group = 'assignment', colour = 'assignment'), data=erupt_curves_optimized_mr) +\
    geom_line()+\
    geom_point()+facet_wrap("response_var_names")

In [11]:
erupt_curves_optimized_mr.iloc[np.where((erupt_curves_optimized_mr['response_var_names'] == 'profit')*(erupt_curves_optimized_mr['assignment'] == 'model'))]

/Users/sweiss/Library/Python/3.8/lib/python/site-packages/pandas-1.2.0-py3.8-macosx-10.9-x86_64.egg/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(


,mean,std,response_var_names,weights,assignment,treatment
3,0.310551,0.010514,profit,"0.0,-1.0,0.0,0.0",model,-1
3,0.334667,0.010415,profit,"0.1,-0.9,0.0,0.0",model,-1
3,0.353287,0.010289,profit,"0.2,-0.8,0.0,0.0",model,-1
3,0.363480,0.010239,profit,"0.30000000000000004,-0.7,0.0,0.0",model,-1
3,0.366594,0.010237,profit,"0.4,-0.6,0.0,0.0",model,-1
3,0.368073,0.010268,profit,"0.5,-0.5,0.0,0.0",model,-1
3,0.358904,0.010355,profit,"0.6000000000000001,-0.3999999999999999,0.0,0.0",model,-1
3,0.353342,0.010447,profit,"0.7000000000000001,-0.29999999999999993,0.0,0.0",model,-1
3,0.342251,0.010522,profit,"0.8,-0.19999999999999996,0.0,0.0",model,-1
3,0.334271,0.010597,profit,"0.9,-0.09999999999999998,0.0,0.0",model,-1


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth = 5, oob_score = True)
rf.fit(x,t)

propensity_scores = rf.predict_proba(x)

propensity_scores

In [ ]:
propensity_scores = pd.DataFrame(1/rf.predict_proba(x))
propensity_scores.columns = rf.classes_

i = pd.Series(t)
propensity_scores.lookup(i.index, i.values)


In [ ]:
rf.oob_decision_function_

In [ ]:
uplift_model_optimized_mr.get_random_erupts()

In [ ]:
    use_propensity = False
    if use_propensity is False:
        propensity_model = None
    


In [ ]:
propensity_model is None